### Price returns in kdb

Return from time $t$ to time ${t+1} is given by:

$$ R_{t,t+1} = \frac{P_{t+1}-P_{t}}{P_{t}} $$

or alternately

$$ R_{t,t+1} = \frac{P_{t+1}}{P_{t}} - 1 $$

In [45]:
/ create some random values as price in the range of 80 - 100

p1: 80+(10?20.0)
p1

92.40603 98.65263 85.49413 81.1505 85.12132 84.62022 81.7448 82.04886 97.3421..


In [46]:
/ calculate the return

/ Use prev methods
\t:10000 (p1 % prev p1) - 1 

/ Use prior
\t:10000 prior[{(x%y)-1};p1]

15


39


In [47]:
/ Use deltas method

\t:10000 (deltas p1) % prev p1

/ Prev method is better than others

8


In [48]:
/ Create table 
t: ([] AAPL: 80+10?20.0;META:80+10?20.0)


/update the CHANGES columns
update AAPL_CHANGES:(AAPL % prev AAPL)-1, META_CHANGES:(META % prev META)-1 from `t;

/update the date sake of date columns

update date:.z.d - reverse til 10 from `t;

/remove the first row
t : 1_t;

t

AAPL     META     AAPL_CHANGES META_CHANGES date      
------------------------------------------------------
99.75569 93.76951 0.130423     0.1262885    2024.09.27
87.73471 96.35509 -0.1205042   0.0275738    2024.09.28
94.53562 95.0402  0.0775168    -0.01364629  2024.09.29
88.09309 82.17365 -0.06814921  -0.1353801   2024.09.30
96.71013 99.19793 0.09781741   0.2071744    2024.10.01
92.85474 80.73367 -0.03986542  -0.1861355   2024.10.02
91.66052 92.86196 -0.01286113  0.150226     2024.10.03
82.84987 93.41748 -0.09612266  0.005982115  2024.10.04
98.29976 93.57816 0.1864806    0.001720122  2024.10.05


In [49]:
/ Get the changes column only and save it in tv variable

tv:: select AAPL_CHANGES, META_CHANGES from t

In [50]:
/ Annual returns in percentage

(prd tv+1)-1

AAPL_CHANGES| 0.1139245
META_CHANGES| 0.1239902


In [51]:
/ Annual return of meta

(prd(1+t`META_CHANGES))-1

0.1239902


# Volatility

`Volatility denotes fluctuation that happens in prices changes. `

`In statistics terms we can call it as standard(Population) deviation.`

In [52]:
t: ("FF";enlist csv)0: read0 `:./data/sample_prices.csv
5#t

BLUE   ORANGE 
--------------
8.7    10.66  
8.9055 11.0828
8.7113 10.71  
8.4346 11.5907
8.7254 12.107 


In [53]:
/ Find the population standard deviation

dev each flip t

BLUE  | 0.3784187
ORANGE| 1.010706


In [54]:
/ Find the returns over period

update BLUE:(BLUE % prev BLUE)-1, ORANGE:(ORANGE % prev ORANGE)-1 from `t;
delete from `t where i =0;
t

BLUE        ORANGE     
-----------------------
0.02362069  0.03966229 
-0.02180675 -0.03363771
-0.03176334 0.08223156 
0.03447704  0.04454433 
0.03778623  -0.02638143
-0.01145211 -0.04918728
0.03267645  0.1170078  
-0.01258127 0.06735255 
0.02958061  0.07824942 
0.006150507 -0.1682607 
0.01216236  0.02404099 
0.02114854  -0.05562346


In [55]:
/ Lets compute the standard deviation

(dev each flip t)

BLUE  | 0.02295663
ORANGE| 0.07621241


In [56]:
/ with formula

avrg: avg t`BLUE

sqrt avg (((t`BLUE) - avrg) xexp 2)

0.02295663


In [57]:
/ Compute the annualize volatility

(dev each flip t)*sqrt(12)

BLUE  | 0.0795241
ORANGE| 0.2640075


# Risk Adjusted Returns

In [58]:
/ Read the csv from source

returns : ("J",19#"F";enlist csv)0: `:./data/Portfolios_Formed_on_ME_monthly_EW.csv

/return header 5 rows

5#returns

x      <= 0   Lo 30 Med 40 Hi 30 Lo 20 Qnt 2 Qnt 3 Qnt 4 Hi 20 Lo 10 Dec 2 De..
-----------------------------------------------------------------------------..
192607 -99.99 -0.43 1.52   2.68  -0.57 0.59  1.6   1.47  3.33  -1.45 0.29  -0..
192608 -99.99 3.9   3.04   2.09  3.84  3.59  3.71  1.61  2.33  5.12  2.59  4...
192609 -99.99 -1.08 -0.54  0.16  -0.48 -1.4  0     -0.5  -0.09 0.93  -1.87 -2..
192610 -99.99 -3.32 -3.52  -3.06 -3.29 -4.1  -2.89 -3.36 -2.95 -4.84 -1.77 -3..
192611 -99.99 -0.46 3.82   3.09  -0.55 2.18  3.41  3.39  3.16  -0.78 -0.32 -0..


In [59]:
/ Chenge the columns name. Kdb doesn't recognize the column name with space

returns : (`$ ssr[;" ";""] each string cols returns) xcol returns
5#returns

x      <=0    Lo30  Med40 Hi30  Lo20  Qnt2 Qnt3  Qnt4  Hi20  Lo10  Dec2  Dec3..
-----------------------------------------------------------------------------..
192607 -99.99 -0.43 1.52  2.68  -0.57 0.59 1.6   1.47  3.33  -1.45 0.29  -0.1..
192608 -99.99 3.9   3.04  2.09  3.84  3.59 3.71  1.61  2.33  5.12  2.59  4.03..
192609 -99.99 -1.08 -0.54 0.16  -0.48 -1.4 0     -0.5  -0.09 0.93  -1.87 -2.2..
192610 -99.99 -3.32 -3.52 -3.06 -3.29 -4.1 -2.89 -3.36 -2.95 -4.84 -1.77 -3.3..
192611 -99.99 -0.46 3.82  3.09  -0.55 2.18 3.41  3.39  3.16  -0.78 -0.32 -0.2..


In [60]:
/ Focus on Lo10 and Hi10 columns

returns: select Lo10, Hi10 from returns

In [61]:
/ Devide the values with 100

returns : returns%100

In [62]:
/ Rename Lo10 as SmallCap and Hi10 as Largecap

returns : `SmallCap`LargeCap xcol returns 

In [63]:
returns

SmallCap LargeCap
-----------------
-0.0145  0.0329  
0.0512   0.037   
0.0093   0.0067  
-0.0484  -0.0243 
-0.0078  0.027   
-0.0266  0.0301  
0.0453   -0.0043 
0.0685   0.0365  
-0.0591  0.0085  
0.0467   0.009   
0.0531   0.0525  
-0.0201  -0.0214 
0.0804   0.0785  
-0.0088  0.0171  
-0.0071  0.0534  
-0.0555  -0.0376 
0.2242   0.0656  
0.0449   0.0073  
0.1679   -0.0116 
-0.0155  -0.0122 
..


In [64]:
/Annualize volatility

annualized_vol: (dev each flip returns)*sqrt(12)
annualized_vol

SmallCap| 0.3680272
LargeCap| 0.1866319


In [65]:
/ Annualize returns per month

((prd(returns + 1)) xexp (1%count returns))-1

SmallCap| 0.01298637
LargeCap| 0.007423424


In [66]:

/ Annualize returns

annualized_return: ((prd(returns + 1)) xexp (12%count returns))-1
annualized_return

SmallCap| 0.1674633
LargeCap| 0.09280968


In [67]:
/ Find the sharpe ratio

/ sharpe ratio is adjusted return with risk free rate and volatility

riskfree_rate : 0.03
excess_return : annualized_return - riskfree_rate
sharpe_ratio : excess_return%annualized_vol
sharpe_ratio

SmallCap| 0.373514
LargeCap| 0.3365432


In [2]:
.p.import[`pandas]

{[f;x]embedPy[f;x]}[foreign]enlist


In [2]:
.p.eval"pandas.DataFrame('a':[1,2,3,5])"

[0;31meval: invalid syntax (, line 1)[0m: [0;31meval: invalid syntax (, line 1)[0m